# 🚀 Day 2-5: 트리 기반 앙상블 모델 (Random Forest & Boosted Tree)

이전 파트에서 우리는 단일 결정 트리(Decision Tree) 모델의 작동 방식과 한계점에 대해 배웠습니다. 

결정 트리는 해석이 쉽고 직관적이지만, 데이터의 작은 변화에도 구조가 크게 바뀔 수 있어 불안정하고 과적합(Overfitting)될 위험이 크다는 단점이 있습니다.

이번 시간에는 이러한 단일 트리의 한계를 극복하고, 예측 성능을 획기적으로 끌어올리는 **앙상블(Ensemble)** 기법에 대해 집중적으로 알아봅니다. 

앙상블은 '여러 개의 약한 모델(Weak Learner)을 결합하여 하나의 강력한 모델(Strong Learner)을 만드는' 아이디어에 기반합니다.

특히, 트리 모델을 기반으로 하는 가장 대표적이고 강력한 앙상블 기법인 **Random Forest** 와 **Boosted Tree (Gradient Boosting 계열)** 를 마스터하는 것을 목표로 합니다.

* **Random Forest**: 여러 개의 결정 트리를 독립적으로 학습시킨 후, 그 결과를 종합(Voting 또는 Averaging)하여 예측 정확도를 높이고 과적합을 방지합니다. **Bagging** 과 **Feature Randomness** 가 핵심 아이디어입니다.
* **Boosted Tree**: 여러 개의 결정 트리를 순차적으로 학습시키며, 이전 단계의 트리가 만든 오류를 다음 단계의 트리가 보완해나가는 방식으로 모델을 점진적으로 개선합니다. **Boosting** 의 대표주자이며, 캐글과 같은 데이터 경진대회에서 최고의 성능을 보여주는 **XGBoost, LightGBM, CatBoost** 가 여기에 속합니다.

이번 실습에서는 `Heart Failure Records` 데이터셋을 사용하여 환자의 생존 여부를 예측하는 분류 문제를 해결하며, 이 강력한 모델들을 어떻게 활용하는지 배워보겠습니다.


---

### 1. 데이터 준비 및 탐색

먼저 실습에 사용할 데이터를 불러오고, 기본 구조와 타겟 변수의 분포를 확인합니다.

* **데이터셋**: Heart Failure Clinical Records (심부전 환자 임상 기록)
* **출처**: Kaggle (ODbL License)
* **목표**: `DEATH_EVENT` (사망 여부) 예측
* **핵심 포인트**: 데이터 불균형(Imbalance) 상태 확인

#### 💻 코드로 알아보기

In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix

# 데이터셋 로드
path = '../datasets/ml/heart-failure/heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(path)

# 데이터 탐색
print("데이터셋 크기:", df.shape)
print("\n결측치 확인:\n", df.isnull().sum())
print("\n데이터 타입 확인:\n", df.info())

# 타겟 변수 분포 시각화
fig = px.pie(df, names='DEATH_EVENT', title='사망 여부(DEATH_EVENT) 분포', hole=0.3)
fig.show()

데이터셋 크기: (299, 13)

결측치 확인:
 age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets         

데이터를 살펴보면, 약 32%의 환자가 사망(DEATH\_EVENT=1)한 것을 알 수 있습니다.

약간의 불균형이 존재하므로, 모델 평가 시 `Accuracy`뿐만 아니라 `F1-Score`나 `ROC-AUC`를 함께 확인하는 것이 중요합니다.


---

### 2. 랜덤 포레스트 (Random Forest)

랜덤 포레스트는 **배깅(Bagging)** 이라는 앙상블 기법에 기반합니다. 

배깅은 **B**ootstrap **agg**regat**ing**의 약자로, 원본 훈련 데이터에서 무작위로 복원추출(Bootstrap)하여 여러 개의 서브 데이터셋을 만들고, 각 서브셋으로 개별 모델(결정 트리)을 학습시킨 후 결과를 종합하는 방식입니다.

#### 🧠 개념 이해하기

* **부트스트랩 샘플링 (Bootstrap Sampling)**: 원본 데이터에서 중복을 허용하여 원본과 같은 크기의 샘플을 여러 개 만드는 과정입니다. 이 과정에서 어떤 데이터는 여러 번 뽑히고, 어떤 데이터는 한 번도 뽑히지 않을 수 있습니다.
  
* **OOB (Out-Of-Bag) Error**: 부트스트랩 샘플에 포함되지 않은 데이터들을 OOB 샘플이라고 합니다. 각 트리를 학습할 때 사용되지 않은 OOB 샘플을 검증 데이터처럼 사용하여 모델 성능을 평가할 수 있습니다. 이는 교차 검증(Cross-Validation)과 비슷한 효과를 냅니다.
* **특성 무작위성 (Feature Randomness)**: 랜덤 포레스트는 각 트리의 노드를 분할할 때 전체 특성 중 일부(e.g., 전체 특성의 제곱근 개수)를 무작위로 선택하고, 그 중에서 최적의 분할을 찾습니다. 이는 각 트리가 서로 다른 특성에 집중하게 만들어 모델의 다양성을 높이고, 트리 간의 상관관계를 줄여 앙상블 효과를 극대화합니다.
* **특성 중요도 (Feature Importance)**: 모델이 어떤 특성을 예측에 중요하게 사용했는지 정량적으로 나타내는 지표입니다. 특정 특성이 노드 분할 시 불순도(Impurity)를 얼마나 감소시켰는지를 기준으로 계산됩니다.

#### 💻 코드로 알아보기: `RandomForestClassifier`

`scikit-learn`의 `RandomForestClassifier`를 사용하여 모델을 학습하고 평가해 보겠습니다.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 데이터 분리
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 파이프라인 구축 (스케일링 + 모델)
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf_clf', RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True))
])

# 모델 학습
rf_pipeline.fit(X_train, y_train)

# OOB 점수 확인
print(f"Random Forest OOB Score: {rf_pipeline.named_steps['rf_clf'].oob_score_:.4f}")

# 예측 및 평가
y_pred_rf = rf_pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, rf_pipeline.predict_proba(X_test)[:, 1]):.4f}")

Random Forest OOB Score: 0.8536
Accuracy: 0.8333
ROC-AUC: 0.8909


#### 💡 특성 중요도 시각화

In [10]:
# 파이프라인에서 모델만 추출
rf_model = rf_pipeline.named_steps['rf_clf']
feature_importances = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# 시각화
fig = px.bar(x=feature_importances.values,
             y=feature_importances.index,
             orientation='h',
             title='Random Forest 특성 중요도',
             labels={'x': 'Importance', 'y': 'Feature'})
fig.show()

`time`(추적 기간), `ejection_fraction`(박출계수), `serum_creatinine`(혈청 크레아티닌)이 사망 예측에 중요한 변수임을 알 수 있습니다.

#### ✏️ 연습문제 1

`RandomForestClassifier`의 `n_estimators`(트리 개수)와 `max_depth`(트리 최대 깊이)를 변경하면서 모델의 성능(정확도, ROC-AUC)이 어떻게 변하는지 확인해보세요.
* Case 1: `n_estimators=200`, `max_depth=5`
* Case 2: `n_estimators=50`, `max_depth=10`

In [11]:
# 코드 작성
# Case 1: n_estimators=200, max_depth=5
rf_case1 = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
rf_case1.fit(X_train, y_train)
y_pred_case1 = rf_case1.predict(X_test)
y_pred_proba_case1 = rf_case1.predict_proba(X_test)[:, 1]

# Case 2: n_estimators=50, max_depth=10
rf_case2 = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
rf_case2.fit(X_train, y_train)
y_pred_case2 = rf_case2.predict(X_test)
y_pred_proba_case2 = rf_case2.predict_proba(X_test)[:, 1]

# 성능 평가
print("Case 1 (n_estimators=200, max_depth=5):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_case1):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_case1):.4f}\n")

print("Case 2 (n_estimators=50, max_depth=10):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_case2):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_case2):.4f}")

Case 1 (n_estimators=200, max_depth=5):
Accuracy: 0.8500
ROC-AUC: 0.8999

Case 2 (n_estimators=50, max_depth=10):
Accuracy: 0.8500
ROC-AUC: 0.8742



---
### 3. 부스팅 트리 (Boosted Tree)

**부스팅(Boosting)** 은 랜덤 포레스트와 달리 트리를 순차적으로 만들어나가는 앙상블 기법입니다.

1.  첫 번째 트리가 데이터를 학습하고 예측합니다.
   
2.  예측 결과와 실제 값의 차이, 즉 **잔차(Residual)** 또는 **오류(Error)**를 계산합니다.
3.  두 번째 트리는 이 **오류를 예측하도록** 학습합니다.
4.  원본 예측값에 (학습률 * 오류 예측값)을 더하여 예측을 보정합니다.
5.  이 과정을 반복하며 모델의 성능을 점진적으로 향상시킵니다.

이러한 부스팅 기법 중 **경사 하강법(Gradient Descent)** 을 사용하여 오류를 보정하는 알고리즘들을 **그래디언트 부스팅(Gradient Boosting)** 이라고 부르며,

XGBoost, LightGBM, CatBoost는 이를 더욱 발전시킨 모델입니다.


#### 3.1 XGBoost (eXtreme Gradient Boosting)

XGBoost는 기존 그래디언트 부스팅 모델(GBM)의 단점인 느린 속도와 과적합 문제를 개선하여 성능과 속도를 모두 잡은 모델입니다.

#### 🧠 개념 이해하기

* **규제 (Regularization)**: 모델의 복잡도를 제어하여 과적합을 방지하는 기능(L1, L2 규제)을 내장하고 있습니다.
  
* **병렬 처리**: 트리 생성 시 각 노드의 분할점을 찾는 과정을 병렬로 처리하여 학습 속도가 매우 빠릅니다.
* **자체 교차 검증 (Built-in Cross-Validation)**: 학습 과정에서 내장된 교차 검증 기능을 사용하여 최적의 트리 개수를 찾을 수 있습니다.
* **결측치 자체 처리**: 데이터에 결측치가 있어도 이를 내부적으로 처리하는 로직이 포함되어 있어 별도의 전처리가 필요 없습니다.

#### 💻 코드로 알아보기: `XGBClassifier`

In [14]:
from xgboost import XGBClassifier

# XGBoost 모델 생성 및 학습
# eval_set과 early_stopping_rounds를 사용하여 조기 종료 기능 활성화
# sklearn pipeline에 XGBClassifier 같이 사용 가능
xgb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb_clf', XGBClassifier(n_estimators=500, learning_rate=0.1, eval_metric='logloss', random_state=42, early_stopping_rounds=50))
])

# 스케일링된 데이터로 eval_set 생성
# xgb_pipeline.named_steps['scaler']. : 스케일러 모델
X_train_scaled = xgb_pipeline.named_steps['scaler'].fit_transform(X_train)
X_test_scaled = xgb_pipeline.named_steps['scaler'].transform(X_test)

# XGBoost 모델 학습
xgb_model = xgb_pipeline.named_steps['xgb_clf']
xgb_model.fit(X_train_scaled, y_train,
              eval_set=[(X_test_scaled, y_test)],
              verbose=False)

# 예측 및 평가
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")

Accuracy: 0.8000
ROC-AUC: 0.8594


*`early_stopping_rounds`*: 검증 세트의 성능이 지정된 라운드 수 동안 개선되지 않으면 학습을 조기 종료하여 과적합을 방지하고 최적의 모델을 찾습니다.

... (이하 LightGBM, CatBoost에 대한 개념, 코드, 연습문제로 이어짐) ...


3.2 LightGBM (Light Gradient Boosting Machine)

LightGBM은 Microsoft에서 개발한 그래디언트 부스팅 프레임워크로, XGBoost보다 더 빠른 학습 속도와 더 적은 메모리 사용량을 특징으로 합니다.

🧠 개념 이해하기

* **Leaf-wise 트리 성장**: 기존의 level-wise 방식과 달리, 가장 손실이 큰 잎 노드를 먼저 분할하는 방식으로 트리를 성장시킵니다.
* **히스토그램 기반 알고리즘**: 연속형 특성을 이산화하여 메모리 사용량을 줄이고 학습 속도를 향상시킵니다.
* **GOSS (Gradient-based One-Side Sampling)**: 그래디언트가 큰 데이터를 우선적으로 사용하여 학습 효율성을 높입니다.
* **EFB (Exclusive Feature Bundling)**: 서로 배타적인 특성들을 하나로 묶어 차원을 줄이는 기술을 사용합니다.

💻 코드로 알아보기: LGBMClassifier

In [15]:
from lightgbm import LGBMClassifier

# LightGBM 모델 생성 및 학습
lgb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lgb_clf', LGBMClassifier(n_estimators=500, learning_rate=0.1, random_state=42))
])

lgb_pipeline.fit(X_train, y_train,
                 lgb_clf__eval_set=[(X_test, y_test)])

# 예측 및 평가
y_pred_lgb = lgb_pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred_lgb):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, lgb_pipeline.predict_proba(X_test)[:, 1]):.4f}")

# 특성 중요도 시각화
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_pipeline.named_steps['lgb_clf'].feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

fig = px.bar(feature_importance, 
             x='importance', 
             y='feature',
             title='LightGBM 특성 중요도',
             orientation='h')
fig.show()


[LightGBM] [Info] Number of positive: 77, number of negative: 162
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 303
[LightGBM] [Info] Number of data points in the train set: 239, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.322176 -> initscore=-0.743791
[LightGBM] [Info] Start training from score -0.743791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


3.3 CatBoost (Categorical Boosting)

CatBoost는 Yandex에서 개발한 그래디언트 부스팅 프레임워크로, 범주형 변수를 자동으로 처리하는 기능이 특징입니다.

🧠 개념 이해하기

* **범주형 변수 처리**: 범주형 변수를 자동으로 인식하고 처리하여 전처리 과정을 단순화합니다.
* **Ordered Boosting**: 과적합을 방지하기 위한 특별한 부스팅 방식입니다.
* **대칭 트리**: 모든 레벨에서 동일한 분할 조건을 사용하여 예측 속도를 향상시킵니다.
* **GPU 가속**: GPU를 활용한 빠른 학습이 가능합니다.

💻 코드로 알아보기: CatBoostClassifier

In [16]:
from catboost import CatBoostClassifier

# CatBoost 모델 생성 및 학습
cat_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('cat_clf', CatBoostClassifier(iterations=500, learning_rate=0.1, random_state=42, verbose=False))
])

cat_pipeline.fit(X_train, y_train)

# 예측 및 평가
y_pred_cat = cat_pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred_cat):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, cat_pipeline.predict_proba(X_test)[:, 1]):.4f}")

# 특성 중요도 시각화
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': cat_pipeline.named_steps['cat_clf'].feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

fig = px.bar(feature_importance, 
             x='importance', 
             y='feature',
             title='CatBoost 특성 중요도',
             orientation='h')
fig.show()


Accuracy: 0.8167
ROC-AUC: 0.8845



---
### 📝 Lab #2 응용 과제: 앙상블 모델 비교 분석

지금까지 배운 랜덤 포레스트, XGBoost, LightGBM, CatBoost 모델을 모두 사용하여 `Heart Failure` 데이터셋에 대한 예측을 수행하고, 어떤 모델이 가장 좋은 성능을 보이는지 비교 분석하는 종합 과제를 수행합니다.

#### 과제 목표

1.  4가지 모델(`RandomForestClassifier`, `XGBClassifier`, `LGBMClassifier`, `CatBoostClassifier`)을 각각 학습시킵니다.
2.  각 모델의 **정확도(Accuracy)**, **F1-Score**, **ROC-AUC** 점수를 계산하여 성능을 비교하는 DataFrame을 생성합니다.
3.  가장 성능이 좋은 모델의 **혼동 행렬(Confusion Matrix)** 을 `Plotly`를 사용하여 시각화하고, 결과를 분석합니다.
4.  가장 성능이 좋은 모델의 **특성 중요도**를 시각화하고, 상위 5개 특성이 무엇인지 설명합니다.

코드작성

In [12]:
# 필요한 라이브러리 임포트
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

# 모델 정의
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(random_state=42, verbose=False)
}

# 결과를 저장할 딕셔너리
results = {}

# 각 모델 학습 및 평가
for name, model in models.items():
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 예측
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 성능 지표 계산
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    }

[LightGBM] [Info] Number of positive: 77, number of negative: 162
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000510 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 297
[LightGBM] [Info] Number of data points in the train set: 239, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.322176 -> initscore=-0.743791
[LightGBM] [Info] Start training from score -0.743791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [17]:
# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results).T
print("모델 성능 비교:")
results_df

모델 성능 비교:


,Accuracy,F1-Score,ROC-AUC
RandomForest,0.833333,0.705882,0.891528
XGBoost,0.816667,0.645161,0.831836
LightGBM,0.833333,0.687500,0.848524
CatBoost,0.800000,0.600000,0.888318


In [18]:
results_df['ROC-AUC'].plot.bar()

In [19]:
# 가장 좋은 성능의 모델 찾기
best_model_name = results_df['ROC-AUC'].idxmax()
best_model = models[best_model_name]
best_model

RandomForestClassifier(random_state=42)

In [20]:
# 최고 성능 모델의 혼동 행렬
y_pred_best = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_best)

# 혼동 행렬 시각화
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['예측: 0', '예측: 1'],
    y=['실제: 0', '실제: 1'],
    text=cm,
    texttemplate='%{text}',
    textfont={"size": 16},
    colorscale='Blues'
))
fig.update_layout(
    title=f'{best_model_name} 모델의 혼동 행렬',
    xaxis_title='예측 레이블',
    yaxis_title='실제 레이블'
)
fig.show()

In [21]:
# 특성 중요도 시각화
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    })
else:
    # CatBoost의 경우
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.get_feature_importance()
    })


In [22]:
feature_importance = feature_importance.sort_values('importance', ascending=False)

# 상위 5개 특성 출력
print("상위 5개 중요 특성:")
print(feature_importance.head())


상위 5개 중요 특성:
              feature  importance
11               time    0.361383
7    serum_creatinine    0.154118
4   ejection_fraction    0.129121
6           platelets    0.076820
0                 age    0.076779


In [23]:
# 특성 중요도 시각화
fig = px.bar(feature_importance.head(10), 
             x='importance', 
             y='feature',
             title=f'{best_model_name} 모델의 특성 중요도 (상위 10개)',
             orientation='h')
fig.show()